## 1. Geração e manipulação da base de dados

### 1.1 Carregamento e estruturação inicial

In [ ]:
# Imnportanto as bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import PercentFormatter
from scipy.stats import zscore
import plotly.express as px
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, make_scorer, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

In [ ]:
# Criando base de dados

np.random.seed(42)

n_linhas = 10000

descrições = ["Alicate","Martelo","Marreta","Cola","Fita","Parafuso","Lanterna"]
cores = ["Azul","Vermelho","Preto","Amarelo","Verde"]
regiões = ["Norte","Sul","Leste","Oeste","Centro-Oeste"]
canais = ["Online","Loja Física"]
campanhas = ["Campanha A","Campanha B","Nenhuma"]
descontos = [0,5,10,15]

df_vendas = pd.DataFrame({
    "Data": pd.date_range("01/01/2025","31/12/2025", periods=n_linhas).strftime("%d/%m/%Y"),
    "Descrição": np.random.choice(descrições,size=n_linhas),
    "Cor": np.random.choice(cores,size=n_linhas),
    "Quantidade": np.random.randint(1,10,size=n_linhas),
    "Valor": np.random.randint(1,100,size=n_linhas),
    "Situação": np.random.choice(["Pago","Não pago"],size=n_linhas),
    "Cliente_ID": np.random.randint(1000,1100,size=n_linhas),
    "Região": np.random.choice(regiões,size=n_linhas),
    "Canal_Venda": np.random.choice(canais,size=n_linhas),
    "Desconto": np.random.choice(descontos,size=n_linhas),
    "Campanha_Marketing": np.random.choice(campanhas,size=n_linhas)
})

# Categoria do produto com base na descrição
categoria_map = {
    "Alicate": "Ferramenta",
    "Martelo": "Ferramenta",
    "Marreta":"Ferramenta",
    "Cola":"Material",
    "Fita":"Material",
    "Parafuso":"Material",
    "Lanterna":"Eletrônico"
}

df_vendas["Categoria_Produto"] = df_vendas["Descrição"].map(categoria_map)

# Valor final com desconto
df_vendas["Valor_Final"] = (df_vendas["Quantidade"] * df_vendas["Valor"]) * (1- df_vendas["Desconto"] / 100)

In [ ]:
#Visualizando o DataFrame
df_vendas.head()

,Data,Descrição,Cor,Quantidade,Valor,Situação,Cliente_ID,Região,Canal_Venda,Desconto,Campanha_Marketing,Categoria_Produto,Valor_Final
0,01/01/2025,Lanterna,Vermelho,4,40,Não pago,1027,Leste,Loja Física,5,Campanha A,Eletrônico,152.0
1,01/01/2025,Cola,Azul,5,6,Não pago,1013,Centro-Oeste,Online,5,Campanha B,Material,28.5
2,01/01/2025,Fita,Vermelho,5,80,Pago,1075,Centro-Oeste,Online,5,Campanha B,Material,380.0
3,01/01/2025,Lanterna,Vermelho,9,96,Pago,1007,Norte,Loja Física,15,Campanha B,Eletrônico,734.4
4,01/01/2025,Marreta,Preto,8,94,Não pago,1006,Sul,Online,15,Campanha A,Ferramenta,639.2


### 1.2 Tratamento de valores nulos

In [ ]:
# Vamos simular alguns valores nulos
df_vendas.loc[np.random.choice(df_vendas.index,size=100,replace=False),"Valor"] = np.nan
df_vendas.loc[np.random.choice(df_vendas.index,size=100,replace=False),"Região"] = np.nan

In [ ]:
# Vamos verificar se consta os valores nulos, observem que no Valor e na Região temos 9.900 não nulos ou seja 100 de cada uma das colunas esta com valores nulos
df_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Data                10000 non-null  object 
 1   Descrição           10000 non-null  object 
 2   Cor                 10000 non-null  object 
 3   Quantidade          10000 non-null  int32  
 4   Valor               9900 non-null   float64
 5   Situação            10000 non-null  object 
 6   Cliente_ID          10000 non-null  int32  
 7   Região              9900 non-null   object 
 8   Canal_Venda         10000 non-null  object 
 9   Desconto            10000 non-null  int32  
 10  Campanha_Marketing  10000 non-null  object 
 11  Categoria_Produto   10000 non-null  object 
 12  Valor_Final         10000 non-null  float64
dtypes: float64(2), int32(3), object(8)
memory usage: 898.6+ KB


In [ ]:
# Vamos tratar os valores nulos trocando os da coluna Valor pela sua mediana e os da coluna Região por "Indefinida"
df_vendas["Valor"].fillna(df_vendas["Valor"].median(),inplace=True)
df_vendas["Região"].fillna("Indefinida",inplace=True)

C:\Users\Otávio Guedes\AppData\Local\Temp\ipykernel_22420\2393086322.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\Otávio Guedes\AppData\Local\Temp\ipykernel_22420\2393086322.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [ ]:
df_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Data                10000 non-null  object 
 1   Descrição           10000 non-null  object 
 2   Cor                 10000 non-null  object 
 3   Quantidade          10000 non-null  int32  
 4   Valor               10000 non-null  float64
 5   Situação            10000 non-null  object 
 6   Cliente_ID          10000 non-null  int32  
 7   Região              10000 non-null  object 
 8   Canal_Venda         10000 non-null  object 
 9   Desconto            10000 non-null  int32  
 10  Campanha_Marketing  10000 non-null  object 
 11  Categoria_Produto   10000 non-null  object 
 12  Valor_Final         10000 non-null  float64
dtypes: float64(2), int32(3), object(8)
memory usage: 898.6+ KB


 ### 1.3 Conversão de tipos e datas

In [ ]:
# Converter a coluna Data para datetime
df_vendas["Data"] = pd.to_datetime(df_vendas["Data"], format="%d/%m/%Y")

In [ ]:
df_vendas["Data"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Data
Non-Null Count  Dtype         
--------------  -----         
10000 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 78.3 KB


###  1.4 Criação de colunas derivadas

In [ ]:
# Vamos criar algumas colunas derivadas

# Ticket médio por cliente (Cria um dataframe)
ticket_medio = df_vendas.groupby("Cliente_ID")["Valor_Final"].mean().round(2).reset_index()
ticket_medio.rename(columns={"Valor_Final":"Ticket_Medio"},inplace=True)

# Utilizando o merge para juntar a coluna Ticket_Medio no DataFrame
df_vendas = df_vendas.merge(ticket_medio, on="Cliente_ID",how="left")